## 1) ML lec 6 -1, 2

### Multinomial classification

In [2]:
import pandas as pd
hours = pd.Series([10, 9, 3, 2, 11])
attendance = pd.Series([5, 5, 2, 4, 1])
grade = pd.Series(['A', 'A', 'B', 'B', 'C'])

In [12]:
df = pd.DataFrame([hours, attendance, grade], index=('hours', 'attendance', 'grade')).T
df

,hours,attendance,grade
0,10,5,A
1,9,5,A
2,3,2,B
3,2,4,B
4,11,1,C


### Hypothesis
#### (1) softmax function

$XW + b=Y$의 결과를 softmax function에 통과 시킨다.(아이디어는 sigmoid와 흡사)

단, softmax funtion은 다음과 같다. 이를 통해 scores는 probability로 볼 수 있게 된다.

Scores가 softmax를 통과한 결과값은 0~1 사이의 값이 되고  
각각의 class에 대한 결과값들을 모두 더하면 확률처럼 1이 된다.

$$ 
S(y_{i}) = \frac{e^{y_{i}}}{\sum_{j} e^{y_{j}}}
$$


#### (2) Cost function: Cross Entropy 정의 

$$ 
Loss = \frac{1}{N} \sum_{i}\; D(S(wx_{i} + b), L_{i})
$$

실제 행렬을 이용한 식으로 구체화 하면

$$ - Y\;log(Y^{-}) $$

이 된다. 여기서 $L_{i}$는  sample data 각각의 label을 나타낸다.

tf에서 위의 hypothesis, cost function을 구현할 때는 다음과 같이 코딩하면 된다.

- hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

- cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis = 1)


Gradient Descent 방법을 사용할 때 
한 step은 다음과 같다.

$$
-alpha *\;gradient\;L(w_{1}, ..., w_{n})
$$

## 2) ML lab 6 - 1

In [29]:
import tensorflow as tf
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

X = tf.placeholder(tf.float32, shape=[None, 4])
y = tf.placeholder(tf.float32, shape=[None, 3])
W = tf.Variable(tf.random_normal([4, 3]), name='weight')
b = tf.Variable(tf.random_normal([1, 3]), name='bias')
yhat = tf.nn.softmax(tf.matmul(X, W) + b)
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(yhat), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(cost)


In [40]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10001):
        _, _cost, _yhat = sess.run([train_op, cost, yhat], 
                              feed_dict={X: x_data, y: y_data})
        if i%1000==0:
            print('step: {}, cost: {}'
                  .format(i, _cost))
    print('---'*25)
    a = sess.run(yhat, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, axis=1)))
    print('---'*25)
    all_new = sess.run(yhat, feed_dict={X: [[1, 11, 7, 9], 
                                            [1, 3, 4, 3],
                                            [1, 1, 0, 1]]})
    print(all_new, sess.run(tf.argmax(all_new, axis=1)))

step: 0, cost: 4.614407539367676
step: 1000, cost: 0.25730690360069275
step: 2000, cost: 0.15895360708236694
step: 3000, cost: 0.11624938249588013
step: 4000, cost: 0.09132903814315796
step: 5000, cost: 0.07508353888988495
step: 6000, cost: 0.0636853575706482
step: 7000, cost: 0.05525968596339226
step: 8000, cost: 0.0487842820584774
step: 9000, cost: 0.04365559294819832
step: 10000, cost: 0.039495013654232025
---------------------------------------------------------------------------
[[  4.29497987e-07   9.99999523e-01   3.33694103e-08]] [1]
---------------------------------------------------------------------------
[[  4.29497987e-07   9.99999523e-01   3.33694103e-08]
 [  9.97642815e-01   2.29586987e-03   6.12828226e-05]
 [  1.04039829e-15   5.67116786e-07   9.99999404e-01]] [1 0 2]


## 2) ML lab 6 - 2
- fancy한 softmax cross entropy with logits!
    - cost = tf.reduce_mean( - tf.reduce_sum(Y * tf.log(hypothesis), axis=1)


- 위 code line 을 좀 더 간편하게(cost function에 hypothesis가 아닌 logits을 넘겨준다는 점에 유의!)
    - logits = tf.matmul(X, W) + b
    - hypothesis = tf.nn.softmax(logits)
    - cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
    - cost = tf.reduce_mean(cost_i) 

In [60]:
import tensorflow as tf
import numpy as np

xy = np.loadtxt('./data/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1]) 
# one hot encoding을 위해서는 datatype이 int32

nb_classes = 7
Y_one_hot = tf.one_hot(Y, nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(cost)

pred = tf.argmax(hypothesis, 1)
correct_pred = tf.equal(pred, tf.argmax(Y_one_hot, 1))
# sample data에 대해 예측한 class(pred)와 실제 one_hot_encoding된 결과가 같은지 판단

acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# 맞게 예측한 sample들을 모두 더히고 그것을 sample data 갯수로 나눈 것


array([[ 1.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  1.,  0.,  1.],
       [ 0.,  0.,  1., ...,  1.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  1.,  0.,  1.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  1.,  0.,  0.]], dtype=float32)

In [67]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2001):
        _, _cost, _acc = sess.run([train_op, cost, acc], feed_dict={X: x_data, Y: y_data})
        if i%500==0:
            print('step: {}, cost: {}, acc: {}'.format(
                i, _cost, _acc))
    _pred = sess.run(pred, feed_dict={X: x_data})
    for p, y in list(zip(_pred, y_data.flatten())):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))
                  
        
    

step: 0, cost: 2.9200494289398193, acc: 0.48514851927757263
step: 500, cost: 0.3231242597103119, acc: 0.9009901285171509
step: 1000, cost: 0.14321300387382507, acc: 0.9603960514068604
step: 1500, cost: 0.08794863522052765, acc: 1.0
step: 2000, cost: 0.06325948983430862, acc: 1.0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 3 True Y: 3
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] 